# Convergence study

For a given campaign load models for all available freqs and plot
convergence, duration ...


In [1]:
# Reload automatically all python modules before each cell exec
%load_ext autoreload
%autoreload 2

In [2]:
# standard python packages
import sys
import time
import os
import pickle
import numpy as np
from model_tools import load_model
# visu
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import scipy.io
import h5py
import simulation_campaigns
import collections
from post_process import compute_errors, plot_campaign, check_time_vectors, plot_errors, plot_y, plot_durations, save_dof, compute_errors_rel
import datetime

## Pick up results file(s)

Campaign parameters are supposed to be saved in a 'pkl' file

Load it and get sets of results (one set = a model run for different values of frequency)

#### List pkl files available:

In [4]:
ls campaign*.pkl

campaign0212.pkl         campaign_new_0612.pkl
campaign_1812.pkl        campaign_short_1312.pkl


#### Load a pkl and print configs and freqs

In [5]:
import pickle
import subprocess
pklfilename = 'campaign_1812.pkl'
#pkl_file = open('campaign_new_0612.pkl', 'rb')
pkl_file = open(pklfilename, 'rb')

remote_data = pickle.load(pkl_file)
for name in remote_data:
    print("Campaign name : " + name)
    if name.find('results') < 0:
        print("Available freqs:")
        print(list(remote_data[name].keys()))
        print("\n")


Campaign name : bass0.0
Available freqs:
['31360', '1003520', '62720', '250880', '125440', '501760', '15680', '4014080', '2007040', '64225280', '16056320', '32112640', '128450560', '8028160', '256901120']


Campaign name : results_paths
Campaign name : bass0.9
Available freqs:
['125440', '250880', '31360', '15680', '62720', '8028160', '2007040', '128450560', '16056320', '32112640', '1003520', '4014080', '64225280', '501760', '256901120']


Campaign name : bass1.0
Available freqs:
['125440', '15680', '250880', '31360', '62720', '501760', '4014080', '8028160', '2007040', '1003520', '32112640', '16056320', '128450560', '64225280', '256901120']


Campaign name : fretless0.0
Available freqs:
['62720', '125440', '31360', '15680', '250880', '501760', '1003520', '2007040', '4014080', '8028160']


Campaign name : fretless0.9
Available freqs:
['4014080', '8028160', '15680', '62720', '31360', '125440', '250880', '2007040', '501760', '1003520']


Campaign name : fretless1.0
Available freqs:
['1568

## Convergence study

In [ ]:
matlab_frets = './bass_guitar/pb2'
matlab_fretless = './fretless_bass_guitar/bsf'
matlab_one_contact = './one_contact/pb1'
errors = {}
freqs = {}
ymin = {}
timers = {}
xref = {}

In [ ]:
figures_path = 'fig_' + os.path.splitext(pklfilename)[0]
if not os.path.exists(figures_path):
    os.mkdir(figures_path)

#names = [name for name in remote_data.keys() if name.find('bass') >=0]
#figures_path = os.path.join(figures_path, 'bass')
#dofs = [500, 853]

dofs = [100, 500, 990]
#names=['bass0.9']
#names = [name for name in remote_data.keys() if name.find('one_contact') >=0]
#figures_path = os.path.join(figures_path, 'one_contact')

names = [name for name in remote_data.keys() if name.find('one_new') >=0]
figures_path = os.path.join(figures_path, 'one_contact')



#
#names = [name for name in remote_data.keys() if name.find('fretless') >=0]
#figures_path = os.path.join(figures_path, 'fretless')

if not os.path.exists(figures_path):
    os.mkdir(figures_path)
print(figures_path)

for name in names:
    campaign = remote_data[name]
    #if name.find('new') >=0 : 
    #    shift = 2
    #else:
    shift=1
    #if name.find('new2') >=0 : 
    #    shift = 1

#    if name.find('bass') >=0 :
#        matlab_input = matlab_frets
 #   elif name.find('fretless') >= 0:
  #      matlab_input = matlab_fretless
   # elif name.find('one_contact') >=0 :
    #    matlab_input = matlab_one_contact
    fileslist = {}
    results_path = remote_data["results_paths"][1]
    timers[name] = np.zeros((len(campaign), 3))
    j = 0
    for freq in campaign:
        fileslist[np.float(freq)] = os.path.join(results_path, campaign[freq][5])
        timers[name][j, :] = [np.float(freq), np.float(campaign[freq][1]), np.int(campaign[freq][2])]
        j += 1                   
    fileslist = collections.OrderedDict(sorted(fileslist.items()))
    #check_time_vectors(filelist, matlab_frets)
    errors[name], ymin[name], freqs[name], xref[name] = compute_errors(fileslist, dofs=dofs, savedofs=name + '_dofs.mat')

In [ ]:
names = [name for name in remote_data.keys() if name.find('bass') >=0]
dof = 853
for name in names:
    campaign = remote_data[name]
    if name.find('bass') >=0 :
        matlab_input = matlab_frets
    elif name.find('fretless') >= 0:
        matlab_input = matlab_fretless
    fileslist = {}
    results_path = remote_data["results_paths"][1]
    timers[name] = np.zeros((len(campaign), 3))
    j = 0
    for freq in campaign:
        fileslist[np.float(freq)] = os.path.join(results_path, campaign[freq][5])
    fileslist = collections.OrderedDict(sorted(fileslist.items()))
    #print(fileslist)
    #check_time_vectors(filelist, matlab_frets)
    matfile = name + '_dof' + str(dof) + '.mat'
    saved_dof =  save_dof(fileslist, dof, matfile)
    

### Plot convergence for bass guitar

In [ ]:
print(errors)
print(figures_path)

In [ ]:
for  name in errors:
    print(name)
    print(len(freqs[name]))
    print(errors[name].shape)

In [ ]:
iplot = 388
for i in range(len(dofs)):
    fig = plot_errors(errors, freqs, i, xref, iplot, figures_path)
    iplot += 1

fig4 = plot_y(ymin, freqs, 4)
##fig4.title()
plt.savefig(os.path.join(figures_path, 'min_y.pdf'))

In [ ]:
yy = ymin['bass0.9']
fr = freqs['bass0.9']
yy.shape

In [ ]:
for i in range(20):
    plt.semilogx(fr, yy[:-1, i])

#### Save results (errors ...) in pkl file

In [ ]:
date = datetime.datetime.now().isoformat()
bckpfilename = os.path.join(figures_path, 'backup' + str(date) + '.pkl')
print('backup to file ' + bckpfilename + ' ...')
bckpfile = open(bckpfilename, 'wb')
backups = (errors, ymin, freqs)
pickle.dump(backups, bckpfile)


In [ ]:
# To load a file:
bckpfile = open('fig_campaign0212/backup2017-12-13T14:26:26.296423.pkl', 'rb')
#bckpfile = open('fig_campaign_new_0612/backup2017-12-14T11:16:22.928082.pkl', 'rb')

results = pickle.load(bckpfile)
err = results[0]
fre = results[2]
bckpfile.close()


for name in errors:
    err[name] = errors[name]
    fre[name] = freqs[name]

### plot simulations durations

In [ ]:
fig = plot_durations(timers)
plt.savefig(os.path.join(figures_path, 'Durations.pdf'))